# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-10-29 14:36:17.519139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-29 14:36:17.519566: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Create complete Model with Custom output Layer

In [2]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [3]:
trainingValues = [0] * 1014 + [1] * 993
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# Test Images not configured
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels= testValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))
print (trainImages)

Found 2007 files belonging to 2 classes.
Found 314 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


## Prediction before training

In [4]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-10-29 14:36:20.049772: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-29 14:36:20.146450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 541ms/step
[[0.5159187]]


## Train Last two layers

In [5]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-10-29 14:36:21.006473: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 22s 322ms/step - loss: 0.6360 - accuracy: 0.8236
Epoch 2/10
63/63 [==============================] - 20s 314ms/step - loss: 0.4607 - accuracy: 0.8814
Epoch 3/10
63/63 [==============================] - 20s 315ms/step - loss: 0.3260 - accuracy: 0.9023
Epoch 4/10
63/63 [==============================] - 20s 314ms/step - loss: 0.2583 - accuracy: 0.9188
Epoch 5/10
63/63 [==============================] - 20s 314ms/step - loss: 0.2195 - accuracy: 0.9317
Epoch 6/10
63/63 [==============================] - 20s 314ms/step - loss: 0.1948 - accuracy: 0.9387
Epoch 7/10
63/63 [==============================] - 20s 314ms/step - loss: 0.1774 - accuracy: 0.9432
Epoch 8/10
63/63 [==============================] - 20s 314ms/step - loss: 0.1629 - accuracy: 0.9472
Epoch 9/10
63/63 [==============================] - 20s 313ms/step - loss: 0.1517 - accuracy: 0.9512
Epoch 10/10
63/63 [==============================] - 20s 313ms/step - loss: 0.1421 - accuracy: 0.9522


## Test Some data

In [6]:
# score = completeModel.evaluate(testImages)
# print ("loss = ", score[0])
# print ("accuracy = ", score[1])

## See what happens with a deer image

In [7]:
from PIL import Image
testPicture = load_img('testData/testDeer/SEQ86955_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 25ms/step
[[0.03808454]]


## Save the model for export

In [15]:
completeModel.save("squirrelModel.keras")

## Check test data for each animal

In [9]:
# import required module
import os
# assign directory
directory = 'testData/testDeer'
 
# iterate over files in
# that directory
# 0 is positives 1 is negativess
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

testData/testDeer/SEQ81957_IMG_0003.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.8608625]]
testData/testDeer/SEQ80916_IMG_0004.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01705135]]
testData/testDeer/SEQ86955_IMG_0001.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.14707306]]
testData/testDeer/SEQ87558_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.0145549]]
testData/testDeer/SEQ81957_IMG_0010.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6750557]]
testData/testDeer/SEQ86931_IMG_0003.JPG___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.8074325]]
testData/testDeer/SEQ81957_IMG_0016.JPG___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.6885794]]
testData/testDeer/SEQ87611_IMG_0006.JPG___crop01_md_v5a.0.0.pt.jpg


In [10]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/SEQ79862_IMG_0012.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.05023559]]
extraTest/testFox/SEQ79856_IMG_0016.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.04858896]]
extraTest/testFox/SEQ79862_IMG_0002.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.1481079]]
extraTest/testFox/SEQ79857_IMG_0012.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.02918266]]
extraTest/testFox/SEQ79869_IMG_0006.JPG___crop00_md_v5a.0.0.ptFlippedImageFlippedImageFlippedImageFlippedImageFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.02405904]]
extraTe

In [11]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 16ms/step
[[0.00836931]]
extraTest/testPossum/2005.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.00300696]]
extraTest/testPossum/2016.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.10569451]]
extraTest/testPossum/2059.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01253358]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01141038]]
extraTest/testPossum/2123.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00715641]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.05888513]]
extraTest/testPossum/2130.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.02950036]]
extraTes

In [12]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/791.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.17933264]]
extraTest/testRabbit/782.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.06660538]]
extraTest/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.02232776]]
extraTest/testRabbit/821.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.41105822]]
extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.27597842]]
extraTest/testRabbit/832.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.19051448]]
extraTest/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.0129033]]
extraTest/testRabbit/784.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01096133]]
extraTest/testRab

In [13]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99115306]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9982552]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9985122]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.9456714]]
extraTest/testSquirrel/91.jpg___crop04_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9982393]]
extraTest/testSquirrel/96.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9997627]]
extraTest/testSquirrel/82.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9970137]]
extraTest/testSquirrel/84.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.998836]]
extraTest/testSq

## Code for data manipulation to creaet more data

In [14]:
# directory = 'trainingData/trainRabbit' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])